## Imports

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os
import re
import csv
import numpy as np
import pandas as pd
import pickle as pkl
from datetime import datetime
from dateutil import parser

import operator
from scipy import stats

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
plt.rcParams['figure.figsize'] = (12.0, 8.0)
plt.rcParams['axes.axisbelow'] = True

from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.io as pio
from plotly.subplots import make_subplots
init_notebook_mode(True)

from hallprobecalib import hpc_ext_path
from hallprobecalib.hpcplots import scatter2d,scatter3d,histo,py_profile

In [5]:
plotdir = hpc_ext_path+'plots/magnet_modeling/linear/'

In [6]:
datadir = "/home/ckampa/Dropbox/LogFiles/"
pkldir = hpc_ext_path+"datafiles/magnet_stability/pkl/"

In [7]:
df = pd.read_pickle(pkldir+"magnet_stability_all.pkl")

## Data processing / cleaning

In [ ]:
regex = re.compile("^2019.*[6-9].\d{2} .*slow\.txt$")
regex2 = re.compile("^2019-1.*.\d{2} .*slow\.txt$")
#regex = re.compile("^2019.*\b1?[0-9]\b.\d{2} .*slow\.txt$")

In [ ]:
files_list = sorted([f for f in os.listdir(datadir) if regex.match(f) or regex2.match(f)])

In [ ]:
# bad but fine for now
files_list.remove('2019-06-12 115902slow.txt')
files_list = np.array(files_list)

In [ ]:
files_list

In [ ]:
len(files_list)

In [ ]:
first = parser.parse('2019-06-12 11:59:02')
last = parser.parse('2019-11-18 12:18:31')
#last = datetime.now()

In [ ]:
first, last

In [ ]:
runtime = last - first
runtime

In [ ]:
runtime.total_seconds()

In [ ]:
sizes_mb = np.array([os.stat(datadir+file).st_size / 1e6 for file in files_list])

In [ ]:
sizes_mb

In [ ]:
sizes_mb.sum()

In [ ]:
# find length of each file
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [ ]:
nlines = np.array([file_len(datadir+file) for file in files_list])

In [ ]:
nlines

In [ ]:
short_files = files_list[nlines < 10]

In [ ]:
short_files

In [ ]:
files_good = files_list[nlines > 10]
sizes_good = sizes_mb[nlines > 10]
nlines_good = nlines[nlines > 10]

In [ ]:
files_good

In [ ]:
# number of data points
nlines_good.sum(), nlines_good.sum() - len(nlines_good)

In [ ]:
npoints = nlines_good.sum() - len(nlines_good) # 1 header line

In [ ]:
npoints

In [ ]:
# active runtime in days
npoints * 90 / 60**2 / 24 

In [ ]:
# uptime
npoints * 90 / runtime.total_seconds()

**Uptime: 73% utilization!**
- Hopefully nearly 100% during calibration

In [ ]:
# get headers
def get_header(fname, N):
    with open(fname) as myfile:
        head = [[s.strip(" ") for s in str.split(next(myfile), ',')] for x in range(N)]
        #for h in head:
        #    h.remove('\n')
        #head = [np.array(h) for h in head]
        #head = [next(myfile).str for x in range(N)]
    if N == 1:
        return head[0]
    else:
        return head

In [ ]:
headers = [get_header(datadir+file, 1) for file in files_good]

In [ ]:
ncols = [len(header) for header in headers]

In [ ]:
#ncols

In [ ]:
for i,header in enumerate(headers):
    for j,item in enumerate(header):
        if item in ["CH13","CH14","CH15","CH16","CH17","CH18","CH19","CH20"]:
            headers[i][j] = "Chamber Wall "+item

In [ ]:
headers_flat = [item for sublist in headers for item in sublist]

In [ ]:
for i,f in enumerate(headers):
    if "Magnet Current [V]" in f:
        print(files_good[i])

In [ ]:
for i,f in enumerate(headers):
    if "Yoke (center magnet)" not in f:
        print(files_good[i])

In [ ]:
unique_cols = sorted(list(set(headers_flat)))

In [ ]:
unique_cols

In [ ]:
for col in unique_cols:
    if ("Raw" in col) or ("Cal" in col):
        print(col)

In [ ]:
df = pd.read_csv(datadir+files_good[-4], names=headers[-4], skiprows=1)

In [ ]:
df.head()

In [ ]:
df.drop('\n',axis=1, inplace=True)

In [ ]:
cols_norm = df.columns

In [ ]:
i = 1
df = pd.read_csv(datadir+files_good[i], names=headers[i], skiprows=1)

In [ ]:
df

In [ ]:
df.drop('\n',axis=1, inplace=True)

In [ ]:
cols_short = df.columns

In [ ]:
sorted(cols_short)

In [ ]:
cols_norm = list(cols_norm)
sorted(cols_norm)

In [ ]:
list(set(cols_norm) - set(cols_short))

In [ ]:
cols_norm.append("run")

In [ ]:
cols_short = list(cols_short)

One file, with a fair amount of data, is missing Zaber info. So leave out for now.

### Load all files into a list of pd.DataFrames

In [ ]:
# trial and error...
#removes = ['Yoke (center magnet)', 'Desk Ambient', 'Outlet Strip Ambient', 
#           'Top fo Orange PS Ambient', 'Spare Ambient on desk',
#           'TrippLite Ambient 3ft high', 'TrippLite Ambient Floor']
#[cols_short.remove(r) for r in removes];

In [ ]:
#cols_short

In [ ]:
df = []
for i,file in enumerate(files_good):
    if ("Yoke (center magnet)" in headers[i]) and ('Zaber_Meas_MicroStep_X' in headers[i]):
        print(f"Processing #{i}: {file}")
        df_ = pd.read_csv(datadir+file, names=headers[i], skiprows=1)
        df_["run"] = i
        #df_.drop('\n',axis=1, inplace=True)
        # convert magnet current...important!
        if "Magnet Current [V]" in df_.columns:
            df_["Magnet Current [A]"] = df_["Magnet Current [V]"] / 10 * 340 # measured voltage / 10 V * 340 A
        df.append(df_[cols_norm])#cols_short])
        #df.append(df_[sorted(df_.columns)])

In [ ]:
# concatenate!
df = pd.concat(df, ignore_index=True,)# sort=False, axis=0)#, axis=0, join='outer', ignore_index=False, keys=None,
              #levels=None, names=None, verify_integrity=False, copy=True)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
dates = [parser.parse(row.Time) for row in df.itertuples()]

In [ ]:
df["Datetime"] = pd.to_datetime(dates)#.astype("str")
df = df.set_index("Datetime")

In [ ]:
df["seconds_delta"] = (df.index - df.index[0]).total_seconds()
df["hours_delta"] = (df.index - df.index[0]).total_seconds()/60**2
df["days_delta"] = (df.index - df.index[0]).total_seconds()/(24*60**2)

In [ ]:
df.head()

In [ ]:
df.to_pickle(pkldir+"magnet_stability_all.pkl")
#df.to_pickle(pkldir+"magnet_stability_all_raw.pkl")

In [ ]:
df = pd.read_pickle(pkldir+"magnet_stability_all.pkl")
#df = pd.read_pickle(pkldir+"magnet_stability_all_raw.pkl")

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
len(df)

In [ ]:
# active runtime in days (GOOD)
len(df) * 90 / 60**2 / 24 

In [ ]:
# uptime (GOOD)
len(df) * 90 / runtime.total_seconds()

**Uptime: 46% utilization (GOOD DATA)!**
- Hopefully nearly 100% during calibration

In [ ]:
len(files_good)

In [ ]:
df.info()

## Analysis

In [8]:
df["2019-09-18":"2019-09-22"].index

DatetimeIndex(['2019-09-18 00:01:26', '2019-09-18 00:02:56',
               '2019-09-18 00:04:26', '2019-09-18 00:05:56',
               '2019-09-18 00:07:26', '2019-09-18 00:08:56',
               '2019-09-18 00:10:26', '2019-09-18 00:11:56',
               '2019-09-18 00:13:26', '2019-09-18 00:14:56',
               ...
               '2019-09-18 09:13:27', '2019-09-18 09:14:57',
               '2019-09-18 09:16:27', '2019-09-18 09:17:57',
               '2019-09-18 09:19:27', '2019-09-18 09:20:57',
               '2019-09-18 09:22:27', '2019-09-18 09:23:57',
               '2019-09-18 09:25:27', '2019-09-18 09:26:57'],
              dtype='datetime64[ns]', name='Datetime', length=378, freq=None)

Notes:
- NMR dismounted 9/21/19, so last chunked data should be anything after that.

In [9]:
#mask_nodmm = (df["NMR [T]"] > 1.) & (df["Magnet Current [A]"] < 10)
mask_nodmm = (df["NMR [T]"] < 1.) | (df["Magnet Current [A]"] < 10)
#mask_time = (df[""])
df_dmm = df[(~mask_nodmm)]
#df_dmm_1 = df_dmm[:"2019-08-01 00:00:00"]
df_dmm_1 = df_dmm[:"2019-08-01 00:00:00"]
#df_dmm_2 = df_dmm["2019-08-01 00:00:00":"2019-09-22 00:00:00"]
df_dmm_2 = df_dmm["2019-08-02 09:00:00":"2019-09-22 00:00:00"]
#df_dmm_2 = pd.concat([df_dmm[:"2019-08-01 00:00:00"], df_dmm["2019-08-15 00:00:00":]])
#df_dmm_2 = df_dmm["2019-08-01 00:00:00":]
df_dmm_3 = df_dmm["2019-09-22 00:00:00":]#["2019-08-01 00:00:00":]

df_dmm_weird = df_dmm["2019-08-01 00:00:00":"2019-08-02 01:00:00"]

## Stuff

In [ ]:
fig = scatter2d(df_dmm_weird.index, df_dmm_weird["NMR [T]"], inline=False)

In [ ]:
fig = scatter2d(df_dmm_1.index, df_dmm_1["NMR [T]"], inline=False)

In [ ]:
fig = scatter2d(3*[df_dmm_1.index],
                [df_dmm_1[col] for col in 
                ["Zaber_Meas_Encoder_X","Zaber_Meas_Encoder_Y","Zaber_Meas_Encoder_Z"]],
                inline=False)

In [ ]:
len(df)

In [ ]:
len(df_dmm)

In [ ]:
df_dmm_1.head(1)

In [ ]:
df_dmm_3.tail(1)

In [ ]:
len(df_dmm_1), len(df_dmm_2), len(df_dmm_3)

In [ ]:
(len(df_dmm_1)+len(df_dmm_2)+len(df_dmm_3)) * 90 / 60 / 60 / 24

In [ ]:
df_dmm_2.index[0:10]

In [ ]:
df_ = df_dmm_weird#2#[:"11-11-19 12:00:00"]#["11-11-19 09:00:00":"11-12-19 9:00:00"]#df["11-09-19 12:00:00":"11-12-19 9:00:00"]
primary_corr = dict()
for col in df_.columns:
    if df_[col].dtype == np.float64:
        slope_v, intercept_v, r_value_v, p_value_v, std_err_v = stats.linregress(df_[col], df_["NMR [T]"])
        #print(col,r_value_v,p_value_v,std_err_v)
        primary_corr[col] = r_value_v

sorted_primary_corr = sorted(primary_corr.items(), key=operator.itemgetter(1))

#[print(s) for s in sorted_primary_corr if ("Yoke" in s[0])];
sorted_primary_corr

In [ ]:
df_dmm.plot.scatter(x="days_delta", y="NMR [T]")

In [ ]:
df_dmm_2["Zaber_Meas_MicroStep_Y"].plot()

In [ ]:
df_dmm_2["Zaber_Meas_MicroStep_Z"].plot()

In [ ]:
df_dmm_2["NMR [T]"].plot()

In [ ]:
df_["NMR [T]"].plot()

In [ ]:
nmr_change = df_dmm["NMR [T]"].diff(periods=1) / df_dmm.seconds_delta.diff(periods=1)

In [ ]:
nmr_change.name = "nmr_rate_of_change"

In [ ]:
nmr_change[nmr_change.notna()]

In [ ]:
plt.scatter(df_dmm.days_delta, nmr_change)
plt.ylim([-0.00025, 0.00025])

In [ ]:
plt.hist(nmr_change[nmr_change.notna()], bins=500);
plt.yscale('log');

In [ ]:
fig = py_profile(x=df_dmm_1["days_delta"], y=df_dmm_1["Magnet Current [A]"], x_bins=20)

In [ ]:
fig = scatter2d(df_[::40].index, df_[::40]["Magnet Current [A]"], df_[::40]["days_delta"], size_list=[5])

In [ ]:
fig = scatter2d(df_dmm_2.index, df_dmm_2["NMR [T]"], df_dmm_2["days_delta"],size_list=[5], inline=False)
#fig = scatter2d(df_[::100].index, df_[::100]["NMR [T]"], df_[::100]["days_delta"])

In [ ]:
fig = scatter2d(df_dmm_2.index, df_dmm_2["Yoke (center magnet)"], df_dmm_2["days_delta"],size_list=[3], inline=False)

In [ ]:
fig = scatter2d(df.index, df["Yoke (center magnet)"], df["days_delta"],size_list=[3], inline=False)

In [ ]:
fig = scatter2d(df.index, df["ICW in"], df["days_delta"],size_list=[3], inline=False)

In [ ]:
fig = scatter2d(df.index, df["Chamber Wall CH16"], df["days_delta"],size_list=[3], inline=False)

In [ ]:
fig = scatter2d(df.index, df["Yoke (center magnet)"], df["NMR [T]"],size_list=[3], inline=False)

In [ ]:
fig = scatter2d(df[::140].index, df[::140]["NMR [T]"], lines=False, markers=True, inline=False)

In [ ]:
fig = scatter2d(df[::140].index, df[::140]["Magnet Current [A]"], lines=False, markers=True, inline=False)

In [ ]:
#mask_nodmm = (df["NMR [T]"] > 1.) & (df["Magnet Current [A]"] < 10)
mask_nodmm = (df["NMR [T]"] < 1.) | (df["Magnet Current [A]"] < 10)

In [ ]:
df_dmm = df[~mask_nodmm]

In [ ]:
len(df_dmm)

In [ ]:
len(df)

In [ ]:
len(df_dmm) / len(df)

In [ ]:
fig = scatter2d(df_dmm[::140].index, df_dmm[::140]["NMR [T]"], lines=False, markers=True, inline=False)
fig = scatter2d(df_dmm[::140].index, df_dmm[::140]["Magnet Current [A]"], lines=False, markers=True, inline=False)

In [ ]:
fig = scatter2d(2*[df_dmm[::140].index],
                [df_dmm[::140]["Magnet Current [A]"]/df_dmm["Magnet Current [A]"].max(),
                 df_dmm[::140]["NMR [T]"]/df_dmm["NMR [T]"].max()],
                lines=False, markers=True, inline=False)

In [ ]:
fig = scatter2d(df_dmm["Magnet Current [A]"], df_dmm["NMR [T]"], df_dmm["days_delta"], inline=False)
fig = scatter2d(df_dmm["Yoke"], df_dmm["NMR [T]"], df_dmm["days_delta"], inline=False)

### Fitting

In [8]:
import scipy.optimize as optimization

In [ ]:
df_ = df_dmm_1

In [ ]:
def field_strength_current_func(DATA, D,):# E):
    CURRENT = DATA
    return D * CURRENT# + E

In [ ]:
guess = np.array([1.2 / 250,])# 1])#, -1e-5, -1e-6, 1.249 / 250,]) #1.249])

In [ ]:
x_dat = df_["Magnet Current [A]"]#df_dmm["Yoke"],df_dmm["Chamber Wall CH16"],df_dmm["Coil 1 (new)"]])

In [ ]:
params, cov = optimization.curve_fit(field_strength_current_func, x_dat, df_["NMR [T]"], guess)
#params2, cov2 = optimization.curve_fit(field_strength_func, x_dat2, df["NMR [T]"], guess)

In [ ]:
D_fit = params[0]
#raspberry yogurt E_fit = params[1]
#A_fit, B_fit, C_fit, D_fit, E_fit = params
#A_fit2, B_fit2, C_fit2, D_fit2 = params2

In [ ]:
params#, params2

In [ ]:
1.2 / 250

In [ ]:
cov#, cov2

In [ ]:
df_["NMR_lsq"] = field_strength_current_func(x_dat, D_fit,)# E_fit) #E_fit)
#df_dmm["NMR_lsq"] = field_strength_func(x_dat, A_fit, B_fit, C_fit, D_fit)
#df_dmm["NMR_lsq"] = field_strength_func(x_dat, A_fit, B_fit, C_fit, D_fit, E_fit)
#df["NMR_lsq2"] = field_strength_func(x_dat2, A_fit2, B_fit2, C_fit2, D_fit2)

In [ ]:
#fig = scatter2d([df_dmm["NMR_lsq"], df_dmm["NMR_lsq2"]], 2*[df["NMR [T]"]], 2*[df.days_delta])

In [ ]:
fig = scatter2d(df_["NMR_lsq"], df_["NMR [T]"], df_.days_delta, ["Viridis"], show_plot=False)
fig.update_xaxes(title="|B| (calculated)")
fig.update_yaxes(title="NMR [T] (measured)")
fig.update_layout(title="Measured vs. Calculated |B|");
plot(fig)

In [ ]:
df_["LSQ_RES_REL"] = (df_["NMR [T]"] - df_["NMR_lsq"]) / df_["NMR [T]"]
#df_["LSQ2_RES_REL"] = (df["NMR [T]"] - df["NMR_lsq2"]) / df["NMR [T]"]
df_["LSQ_RES"] = (df_["NMR [T]"] - df_["NMR_lsq"])
#f["LSQ2_RES"] = (df["NMR [T]"] - df["NMR_lsq2"])

In [ ]:
fig = scatter2d(df_["NMR_lsq"], df_["LSQ_RES_REL"], df_.days_delta, ["Viridis"], show_plot=False)
plot(fig)

In [ ]:
df_ = df_dmm_weird#[:"11-11-19 12:00:00"]#["11-11-19 09:00:00":"11-12-19 9:00:00"]#df["11-09-19 12:00:00":"11-12-19 9:00:00"]
primary_corr = dict()
for col in df_.columns:
    if df_[col].dtype == np.float64:
        slope_v, intercept_v, r_value_v, p_value_v, std_err_v = stats.linregress(df_[col], df_["NMR_lsq"])
        #print(col,r_value_v,p_value_v,std_err_v)
        primary_corr[col] = r_value_v

sorted_primary_corr = sorted(primary_corr.items(), key=operator.itemgetter(1))

#[print(s) for s in sorted_primary_corr if ("Yoke" in s[0])];
sorted_primary_corr

In [ ]:
filename = "NMR_measured_vs_calculated"
pio.write_image(fig, plotdir+filename+".pdf")
pio.write_image(fig, plotdir+filename+".png")

In [ ]:
fig = scatter2d(df_dmm["Yoke"], df_dmm["NMR_lsq"], df_dmm["days_delta"], inline=False)

In [ ]:
df_ = df_dmm_1#_2

In [ ]:
df_["Zab_Dist"] = (df_["Zaber_Meas_MicroStep_X"]**2 + df_["Zaber_Meas_MicroStep_Y"]**2 + df_["Zaber_Meas_MicroStep_Z"]**2)**(1/2)

## FMS #61

### New Variables

- Idea: new var (Yoke - Chamber CH16)

In [13]:
df_dmm_3

,Time,Magnet Current [A],Magnet Voltage [V],Coil 1 (new),Coil 2 (old),LCW in Coil1,LCW out Coil 1,LCW in Coil 2,LCW out Coil 2,Yoke,...,Zaber_Meas_MicroStep_X,Zaber_Meas_Encoder_X,Zaber_Meas_MicroStep_Y,Zaber_Meas_Encoder_Y,Zaber_Meas_MicroStep_Z,Zaber_Meas_Encoder_Z,run,seconds_delta,hours_delta,days_delta
Datetime,,,,,,,,,,,,,,,,,,,,,
2019-11-08 12:33:49,11/8/2019 12:33:49 PM,256.074400,31.276900,35.151032,34.447445,19.959066,21.814058,22.372099,24.462584,24.933077,...,249952.34375,639878.0,750000.0,1920000.0,750000.0,1920000.0,20,9944469.0,2762.352500,115.098021
2019-11-08 12:35:19,11/8/2019 12:35:19 PM,256.074400,31.432483,36.439224,35.549778,19.981903,21.998850,22.456932,24.887590,24.905584,...,249952.34375,639878.0,750000.0,1920000.0,750000.0,1920000.0,20,9944559.0,2762.377500,115.099063
2019-11-08 12:36:49,11/8/2019 12:36:49 PM,256.074400,31.567833,37.553742,36.548729,20.034805,22.191668,22.610453,25.243853,24.871288,...,249952.34375,639878.0,750000.0,1920000.0,750000.0,1920000.0,20,9944649.0,2762.402500,115.100104
2019-11-08 12:38:19,11/8/2019 12:38:19 PM,256.074400,31.685683,38.551891,37.365566,20.093031,22.362549,22.723967,25.570019,24.844877,...,249952.34375,639878.0,750000.0,1920000.0,750000.0,1920000.0,20,9944739.0,2762.427500,115.101146
2019-11-08 12:39:49,11/8/2019 12:39:49 PM,256.074400,31.789067,39.414150,38.138889,20.141634,22.508911,22.888407,25.866650,24.815359,...,249952.34375,639878.0,750000.0,1920000.0,750000.0,1920000.0,20,9944829.0,2762.452500,115.102187
2019-11-08 12:41:19,11/8/2019 12:41:19 PM,256.074513,31.879683,40.198971,38.786617,20.185642,22.637867,22.971603,26.102793,24.791487,...,249952.34375,639878.0,750000.0,1920000.0,750000.0,1920000.0,20,9944919.0,2762.477500,115.103229
2019-11-08 12:42:49,11/8/2019 12:42:49 PM,256.074513,31.959100,40.846626,39.348164,20.211172,22.752817,23.062761,26.318728,24.775812,...,249952.34375,639878.0,750000.0,1920000.0,750000.0,1920000.0,20,9945009.0,2762.502500,115.104271
2019-11-08 12:44:19,11/8/2019 12:44:19 PM,256.074740,32.028283,41.382408,39.787960,20.184837,22.803463,23.096254,26.491234,24.773256,...,249952.34375,639878.0,750000.0,1920000.0,750000.0,1920000.0,20,9945099.0,2762.527500,115.105312
2019-11-08 12:45:49,11/8/2019 12:45:49 PM,256.074570,32.087417,41.831230,40.215427,20.086845,22.786444,23.122650,26.559860,24.733435,...,249952.34375,639878.0,750000.0,1920000.0,750000.0,1920000.0,20,9945189.0,2762.552500,115.106354


In [22]:
df_ = df_dmm_3

In [23]:
df_["Chamber Wall CH16"]

Datetime
2019-11-08 12:33:49    22.355905
2019-11-08 12:35:19    22.082123
2019-11-08 12:36:49    21.848816
2019-11-08 12:38:19    21.620279
2019-11-08 12:39:49    21.399735
2019-11-08 12:41:19    21.210203
2019-11-08 12:42:49    21.018383
2019-11-08 12:44:19    20.835197
2019-11-08 12:45:49    20.666006
2019-11-08 12:47:20    20.478872
2019-11-08 12:48:49    20.318340
2019-11-08 12:50:19    20.157875
2019-11-08 12:51:49    20.014677
2019-11-08 12:53:19    19.838419
2019-11-08 12:54:49    19.682943
2019-11-08 12:56:19    19.552706
2019-11-08 12:57:49    19.421963
2019-11-08 12:59:19    19.292383
2019-11-08 13:00:49    19.166420
2019-11-08 13:02:19    19.043663
2019-11-08 13:03:49    18.928722
2019-11-08 13:05:19    18.823601
2019-11-08 13:06:49    18.717110
2019-11-08 13:08:19    18.614370
2019-11-08 13:09:49    18.517816
2019-11-08 13:11:19    18.429588
2019-11-08 13:12:49    18.355120
2019-11-08 13:14:19    18.269133
2019-11-08 13:15:49    18.201597
2019-11-08 13:17:19    18.152611
 

In [24]:
fig = scatter2d(2*[df_.index],[df_[col] for col in ["Yoke (center magnet)","Chamber Wall CH16"]], inline=False)

In [25]:
df_["Yoke_minus_CH16"] = df_["Yoke (center magnet)"] - df_["Chamber Wall CH16"]

In [26]:
fig = scatter2d([df_.index],[df_[col] for col in ["Yoke_minus_CH16"]], inline=False)

In [27]:
fig = scatter2d(df_.Yoke_minus_CH16, df_["NMR [T]"], df_.days_delta, inline=False)

In [28]:
fig = scatter2d(df_["Yoke (center magnet)"], df_["NMR [T]"], df_.days_delta, inline=False)

In [29]:
import scipy.optimize as optimization

In [61]:
def field_strength_func(DATA, A, B, C, D,): #E):
    CURRENT, YOKE, ENC, WATER = DATA
    return 1* \
            A * YOKE \
          + C * WATER \
          + D * CURRENT# + E
#+ B * ENC \

In [62]:
guess = np.array([-1e-4, -1e-5, -1e-6, 1.249 / 250,]) #1.249])

In [63]:
NMR = []
NMR_lsq = []
DAYS = []
params_dict = {}
params2_dict = {}
figs = []

for df_ in [df_[200:]]:
    x_dat = np.vstack([df_["Magnet Current [A]"],df_["Yoke (center magnet)"],df_["Chamber Wall CH16"],df_["LCW in Coil 2"]])
    x_dat2 = np.vstack([df_["Magnet Current [A]"],df_["Yoke_minus_CH16"],df_["Floor"],df_["LCW in Coil 2"]])
    params, cov = optimization.curve_fit(field_strength_func, x_dat, df_["NMR [T]"], guess)
    params2, cov2 = optimization.curve_fit(field_strength_func, x_dat2, df_["NMR [T]"], guess)
    params_dict[str(df_.index[0])] = params
    params2_dict[str(df_.index[0])] = params2
    A_fit, B_fit, C_fit, D_fit = params
    #A_fit, B_fit, C_fit, D_fit, E_fit = params
    A_fit2, B_fit2, C_fit2, D_fit2 = params2
    df_["NMR_lsq"] = field_strength_func(x_dat, A_fit, B_fit, C_fit, D_fit)
    #df_dmm["NMR_lsq"] = field_strength_func(x_dat, A_fit, B_fit, C_fit, D_fit, E_fit)
    df_["NMR_lsq2"] = field_strength_func(x_dat2, A_fit2, B_fit2, C_fit2, D_fit2)
    NMR.append(df_["NMR [T]"])
    NMR_lsq.append(df_["NMR_lsq"])
    DAYS.append(df_.days_delta)
    #fig = scatter2d([df_["NMR_lsq"],df_["NMR_lsq2"]], 2*[df_["NMR [T]"]], 2*[df_.days_delta],
    #            ["Viridis", "agsunset"], size_list=2*[5], inline=False, show_plot=False);
    fig = scatter2d([df_["NMR_lsq"],df_["NMR_lsq2"]], 2*[df_["NMR [T]"]], 2*[df_.days_delta],
                ["Viridis", "Jet"], size_list=2*[5], inline=False, show_plot=False);
    fig.update_layout(title=f"Start: {str(df_.index[0])}, End: {str(df_.index[-1])}")
    d = [df_["NMR [T]"].min(), df_["NMR [T]"].max()]
    fig.add_trace(go.Scatter(
                    x=d, y=d,
                    mode='lines', name="unity",
                    line=dict(color='deeppink', width=2,
                              dash='dash')))
    #pio.write_image(fig, plotdir+f"nmr_vs_nmrlsq_{str(df_.index[0])}.pdf")
    plot(fig)
    figs.append(fig)

/home/ckampa/anaconda3/envs/Mu2E/lib/python3.7/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning:

Covariance of the parameters could not be estimated



In [64]:
params_dict

{'2019-11-10 02:03:51': array([-5.89478852e-05, -1.00000000e-05,  4.92218470e-06,  4.88424420e-03])}

In [65]:
params2_dict

{'2019-11-10 02:03:51': array([-5.02200387e-05, -1.00000000e-05, -1.66778196e-05,  4.88191123e-03])}

In [12]:
NMR = pd.concat(NMR, ignore_index=True,)
NMR_lsq = pd.concat(NMR_lsq, ignore_index=True,)
DAYS = pd.concat(DAYS, ignore_index=True,)

In [13]:
fig = scatter2d([NMR_lsq], [NMR], [DAYS],
                ["Viridis"], size_list=[6], inline=False, show_plot=False);
fig.update_layout(title=f"All Data")
d = [NMR_lsq.min(), NMR_lsq.max()]
fig.add_trace(go.Scatter(
            x=d, y=d,
            mode='lines', name="unity",
            line=dict(color='deeppink', width=2,
            dash='dash')))
plot(fig)

'temp-plot.html'

### In Slides v1

In [10]:
import scipy.optimize as optimization

In [11]:
def field_strength_func(DATA, A, B, C, D,): #E):
    CURRENT, YOKE, ENC, WATER = DATA
    return A * YOKE + B * ENC + C * WATER + D * CURRENT# + E

In [12]:
guess = np.array([-1e-4, -1e-5, -1e-6, 1.249 / 250,]) #1.249])

In [15]:
NMR = []
NMR_lsq = []
DAYS = []
params_dict = {}
params2_dict = {}
figs = []

for df_ in [df_dmm_1, df_dmm_2, df_dmm_3, df_dmm_weird]:
    x_dat = np.vstack([df_["Magnet Current [A]"],df_["Yoke (center magnet)"],df_["Chamber Wall CH16"],df_["LCW in Coil 2"]])
    x_dat2 = np.vstack([df_["Magnet Current [A]"],df_["Yoke (center magnet)"],df_["Floor"],df_["LCW in Coil 2"]])
    params, cov = optimization.curve_fit(field_strength_func, x_dat, df_["NMR [T]"], guess)
    params2, cov2 = optimization.curve_fit(field_strength_func, x_dat2, df_["NMR [T]"], guess)
    params_dict[str(df_.index[0])] = params
    params2_dict[str(df_.index[0])] = params2
    A_fit, B_fit, C_fit, D_fit = params
    #A_fit, B_fit, C_fit, D_fit, E_fit = params
    A_fit2, B_fit2, C_fit2, D_fit2 = params2
    df_["NMR_lsq"] = field_strength_func(x_dat, A_fit, B_fit, C_fit, D_fit)
    #df_dmm["NMR_lsq"] = field_strength_func(x_dat, A_fit, B_fit, C_fit, D_fit, E_fit)
    df_["NMR_lsq2"] = field_strength_func(x_dat2, A_fit2, B_fit2, C_fit2, D_fit2)
    NMR.append(df_["NMR [T]"])
    NMR_lsq.append(df_["NMR_lsq"])
    DAYS.append(df_.days_delta)
    #fig = scatter2d([df_["NMR_lsq"],df_["NMR_lsq2"]], 2*[df_["NMR [T]"]], 2*[df_.days_delta],
    #            ["Viridis", "agsunset"], size_list=2*[5], inline=False, show_plot=False);
    fig = scatter2d([df_["NMR_lsq2"]], [df_["NMR [T]"]], [df_.days_delta],
                ["Viridis"], size_list=[5], inline=False, show_plot=False);
    fig.update_layout(title=f"Start: {str(df_.index[0])}, End: {str(df_.index[-1])}")
    d = [df_["NMR [T]"].min(), df_["NMR [T]"].max()]
    fig.add_trace(go.Scatter(
                    x=d, y=d,
                    mode='lines', name="unity",
                    line=dict(color='deeppink', width=2,
                              dash='dash')))
    #pio.write_image(fig, plotdir+f"nmr_vs_nmrlsq_{str(df_.index[0])}.pdf")
    plot(fig)
    figs.append(fig)

In [12]:
NMR = pd.concat(NMR, ignore_index=True,)
NMR_lsq = pd.concat(NMR_lsq, ignore_index=True,)
DAYS = pd.concat(DAYS, ignore_index=True,)

In [13]:
fig = scatter2d([NMR_lsq], [NMR], [DAYS],
                ["Viridis"], size_list=[6], inline=False, show_plot=False);
fig.update_layout(title=f"All Data")
d = [NMR_lsq.min(), NMR_lsq.max()]
fig.add_trace(go.Scatter(
            x=d, y=d,
            mode='lines', name="unity",
            line=dict(color='deeppink', width=2,
            dash='dash')))
plot(fig)

'temp-plot.html'

In [14]:
pio.write_image(fig, plotdir+f"nmr_vs_nmrlsq_all.pdf")

In [17]:
for i,fig in enumerate(figs):
    pio.write_image(fig, plotdir+f"nmr_vs_nmrlsq_{i}.pdf")

In [ ]:
params_dict

In [ ]:
params2_dict

In [ ]:
df_params = pd.DataFrame(params2_dict,)# columns=["A","B","C","D",])

In [ ]:
df_params = df_params.T

In [ ]:
df_params.columns = ["A","B","C","D",]

In [ ]:
df_params["Start Time"] = df_params.index

In [ ]:
df_params = df_params[["Start Time", "A", "B", "C", "D"]]

In [ ]:
df_params

In [ ]:
latex_table = df_params.to_latex(float_format="{:0.2E}".format, index=False)

In [ ]:
print(latex_table)

In [ ]:
x_dat = np.vstack([df_["Magnet Current [A]"],df_["Yoke"],df_["Floor"],df_["LCW in Coil 2"]])
x_dat2 = np.vstack([df_["Magnet Current [A]"],df_["PS Vent1"],df_["Floor"],df_["LCW in Coil 2"]])

In [ ]:
x_dat[2]

In [ ]:
field_strength_func(x_dat, -1e-4, -1e-5, -1e-6, 1.249 / 250,)# 1.2)

In [ ]:
params, cov = optimization.curve_fit(field_strength_func, x_dat, df_["NMR [T]"], guess)
params2, cov2 = optimization.curve_fit(field_strength_func, x_dat2, df_["NMR [T]"], guess)

In [ ]:
A_fit, B_fit, C_fit, D_fit = params
#A_fit, B_fit, C_fit, D_fit, E_fit = params
A_fit2, B_fit2, C_fit2, D_fit2 = params2

In [ ]:
params, params2

In [ ]:
cov, cov2

In [ ]:
df_["NMR_lsq"] = field_strength_func(x_dat, A_fit, B_fit, C_fit, D_fit)
#df_dmm["NMR_lsq"] = field_strength_func(x_dat, A_fit, B_fit, C_fit, D_fit, E_fit)
df_["NMR_lsq2"] = field_strength_func(x_dat2, A_fit2, B_fit2, C_fit2, D_fit2)

In [ ]:
#fig = scatter2d([df_dmm["NMR_lsq"], df_dmm["NMR_lsq2"]], 2*[df["NMR [T]"]], 2*[df.days_delta])

In [ ]:
fig = scatter2d([df_["NMR_lsq"],df_["NMR_lsq2"]], 2*[df_["NMR [T]"]], 2*[df_.days_delta],
                ["Viridis", "agsunset"], size_list=2*[5], inline=False, show_plot=False);
fig.update_layout(title=f"Start: {str(df_.index[0])}, End: {str(df_.index[0])}")
plot(fig)

In [ ]:
fig = scatter2d(df_["NMR_lsq"], df_["NMR [T]"], df_.days_delta, ["Viridis"], show_plot=False)
fig.update_xaxes(title="|B| (calculated)")
fig.update_yaxes(title="NMR [T] (measured)")
fig.update_layout(title="Measured vs. Calculated |B|");
plot(fig)

In [ ]:
filename = "NMR_measured_vs_calculated"
pio.write_image(fig, plotdir+filename+".pdf")
pio.write_image(fig, plotdir+filename+".png")

In [ ]:
df_["LSQ_RES_REL"] = (df_["NMR [T]"] - df_["NMR_lsq"]) / df_["NMR [T]"]
df_["LSQ2_RES_REL"] = (df_["NMR [T]"] - df_["NMR_lsq2"]) / df_["NMR [T]"]
df_["LSQ_RES"] = (df_["NMR [T]"] - df_["NMR_lsq"])
df_["LSQ2_RES"] = (df_["NMR [T]"] - df_["NMR_lsq2"])

In [ ]:
fig = scatter2d(df_["Zab_Dist"], df_["LSQ_RES"], df_["days_delta"], inline=False)

In [ ]:
fig = scatter2d(df_["Zab_Dist"], df_["NMR [T]"], df_["days_delta"], inline=False)

In [ ]:
fig = scatter2d(df_["NMR_lsq2"], df_["LSQ2_RES_REL"], df_.days_delta, ["Viridis"], show_plot=True, inline=False)

In [ ]:
fig.update_traces(marker=dict(size=5))

In [ ]:
fig = scatter2d(df.index, df["LSQ2_RES_REL"], df.days_delta, ["Viridis"], show_plot=True)

In [ ]:
fig.update_traces(marker=dict(size=5))

In [ ]:
fig = histo(df["LSQ2_RES"], bins=100, show_plot=False)# opacity=0.7)
fig.update_xaxes(title="( NMR_Meas - |B|(calculated) )")
fig.update_layout(title="|B| Calculation Residuals")

In [ ]:
fig = histo(df["LSQ2_RES_REL"], bins=100, show_plot=False)# opacity=0.7)
fig.update_xaxes(title="( NMR_Meas - |B|(calculated) ) / NMR_Meas")
fig.update_layout(title="|B| Calculation Relative Residuals")

In [ ]:
filename = "NMR_calculation_res_hist"
pio.write_image(fig, plotdir+filename+".pdf")
pio.write_image(fig, plotdir+filename+".png")

In [ ]:
fig = histo([df["LSQ_RES_REL"],df["LSQ2_RES_REL"],], bins=75, opacity=0.7)

In [ ]:
import scipy.stats as stats

In [ ]:
chi2 = np.sum(np.square((df["NMR [T]"] - df["NMR_lsq2"])/2.9e-6))

In [ ]:
chi2

In [ ]:
dof = len(df) - 4 # 4 param fit
chis = np.linspace(0, 10000, 100000)
pdf = stats.chi2.pdf(chis, dof)

In [ ]:
dof

In [ ]:
chi2 / dof

In [ ]:
plt.plot(chis, pdf, label=r"p.d.f., $\nu = 2$")

In [ ]:
dof

plt.plot(chis/dof, pdf, label=r"p.d.f., $\nu = 2$")
plt.plot([chi2/dof, chi2/dof],[0, (pdf).max()], "r--", label= rf"$\chi^2_{{min}} =${chi2/dof:.3f}")
plt.xlabel(r"$\chi^2$")
plt.ylabel("p.d.f.")
plt.legend();

### Old

In [ ]:
# remove empty runs
files_list.remove('2019-06-19 075455slow.txt')
files_list.remove('2019-07-02 112857slow.txt')

files_list.remove('2019-07-02 150753slow.txt') # unsure...

In [ ]:
files_list

In [ ]:
len(files_list)

In [ ]:
df = []
df_full = []
durations = []

for i,f in enumerate(files_list):
    df_ = pd.read_csv(directory+f)
    df_full_ = pd.read_csv(directory+f)
    run = i+1
    df_["run"] = run
    df_full_["run"] = run
    print(len(df_))
    dates = [parser.parse(row.Time) for row in df_.itertuples()]
    df_["Datetime"] = pd.to_datetime(dates)#.astype("str")
    df_ = df_.set_index("Datetime")
    df_full_["Datetime"] = pd.to_datetime(dates)#.astype("str")
    df_full_ = df_full_.set_index("Datetime")
    
    df_ = df_[df_[" NMR [T]"] != 0]
    # remove outliers
    df_["outlier"] = (df_[" NMR [T]"] < df_[" NMR [T]"].mean()-10*df_[" NMR [T]"].std()) | (df_[" NMR [T]"] > df_[" NMR [T]"].mean()+10*df_[" NMR [T]"].std())
    df_ = df_[df_["outlier"] == False]
    
    durations.append(df_.index[-1] - df_.index[0])
    print(f"File: {f}, Duration: {durations[-1]}, Mean: {df_[' NMR [T]'].mean()}, Std: {df_[' NMR [T]'].std()}")
    df.append(df_)
    df_full.append(df_full_)
    
    df_full_.to_pickle(pkldir+f"run{i+1:02d}_full.pkl")
    df_.to_pickle(pkldir+f"run{i+1:02d}.pkl")

In [ ]:
df[11-1] = df[11-1].loc['2019-08-02 05:00:00':]

In [ ]:
df[11-1].to_pickle(pkldir+f"run11.pkl")

In [ ]:
# test pickled files
df = []
df_full = []
durations = []

for i in range(len(files_list)):
    f = f"run{i+1:02d}"
    df_ = pd.read_pickle(pkldir+f+".pkl")
    df_full_ = pd.read_pickle(pkldir+f+"_full.pkl")
    print(len(df_full_))
    #dates = [parser.parse(row.Time) for row in df_.itertuples()]
    #df_["Datetime"] = pd.to_datetime(dates)#.astype("str")
    #df_ = df_.set_index("Datetime")
    #df_full_["Datetime"] = pd.to_datetime(dates)#.astype("str")
    #df_full_ = df_full_.set_index("Datetime")
    
    #df_ = df_[df_[" NMR [T]"] != 0]
    durations.append(df_.index[-1] - df_.index[0])
    print(f"File: {f}, Duration: {durations[-1]}, Mean: {df_[' NMR [T]'].mean()}, Std: {df_[' NMR [T]'].std()}")
    df.append(df_)
    df_full.append(df_full_)
    
    #df_full_.to_pickle(pkldir+f"run{i+1:02d}_full.pkl")
    #df_.to_pickle(pkldir+f"run{i+1:02d}.pkl")

In [ ]:
files_dict = {f"run{i+1:02d}": f for i,f in enumerate(files_list)}

In [ ]:
files_dict

In [ ]:
csv_columns = ["Run Name", "Filename"]

In [ ]:
csv_file = hpc_ext_path+"datafiles/magnet_stability/runs_list.csv"

In [ ]:
with open(csv_file, "w") as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(csv_columns)
    for row,data in files_dict.items():
        writer.writerow([row,data])

In [ ]:
for row,data in files_dict.items():
    print ({row:data})

## Analyze!

In [ ]:
for df_ in df:
    df_[" NMR [T]"].plot()

plt.legend()

In [ ]:
legends = [f"run{df_.iloc[0].run:02d}" for df_ in df]
fig = scatter2d([df_.index for df_ in df], [df_["NMR_const"] for df_ in df], legend_list=legends, inline=True)

In [ ]:
fname = "NMR_const_all_runs"
pio.write_image(fig,plotdir+f"{fname}.pdf")
pio.write_image(fig,plotdir+f"{fname}.png")

In [ ]:
legends = [f"run{df_.iloc[0].run:02d}" for df_ in df]
fig = scatter2d([df_.index for df_ in df], [df_[" NMR [T]"] for df_ in df], legend_list=legends, inline=True)

In [ ]:
fname = "NMR_all_runs"
pio.write_image(fig,plotdir+f"{fname}.pdf")
pio.write_image(fig,plotdir+f"{fname}.png")

In [ ]:
df[9].head()

In [ ]:
df[4].tail()

In [ ]:
for df_, duration in zip(df, durations):
    print(len(df_), duration.days*24*60 + duration.seconds/60)

This shows that only the last dataset is messy....should probably just cut off the first chunk of that dataset.

In [ ]:
for i,df_ in enumerate(df):
    #outliers_mask = (df_[" NMR [T]"] < df_[" NMR [T]"].mean()-10*df_[" NMR [T]"].std()) | (df_[" NMR [T]"] > df_[" NMR [T]"].mean()+10*df_[" NMR [T]"].std())
    df_["outlier"] = (df_[" NMR [T]"] < df_[" NMR [T]"].mean()-10*df_[" NMR [T]"].std()) | (df_[" NMR [T]"] > df_[" NMR [T]"].mean()+10*df_[" NMR [T]"].std())
    print(f"run{df_.iloc[10].run:02d}: {df_['outlier'].sum()}")
    df_ = df_[df_["outlier"] == False]
    df[i] = df_
    print(f"run{df_.iloc[10].run:02d}: {df_['outlier'].sum()}")

In [ ]:
num_points = []

for df_ in df:
    num_points.append(len(df_))

In [ ]:
sum(num_points)

In [ ]:
total_minutes = []
for d in durations:
    total_minutes.append(d.days * 24 * 60 + d.seconds / 60)

In [ ]:
sum(total_minutes)/60/24

In [ ]:
sum(total_minutes)/60

In [ ]:
sum(total_minutes)

### Correlations

In [ ]:
for i,df_ in enumerate(df):
    primary_corr = dict()
    for col in df_.columns:
        if df_[col].dtype == np.float64:
            slope_v, intercept_v, r_value_v, p_value_v, std_err_v = stats.linregress(df_[col], df_[" NMR [T]"])
            #print(col,r_value_v,p_value_v,std_err_v)
            primary_corr[col] = [r_value_v, slope_v]
    sorted_primary_corr = sorted(primary_corr.items(), key=operator.itemgetter(1))
    
    _ = sorted_primary_corr[0]
    df[i]["corr_var"] = _[0]
    print(f"run{df_.iloc[0].run:02d}: {sorted_primary_corr[0]}")#,sorted_primary_corr[-3:]}")

In [ ]:
df[0].iloc[0].corr_var

In [ ]:
for i,df_ in enumerate(df):
    corr_main = df_.iloc[0].corr_var
    slope, intercept, r_value, p_value, std_err = stats.linregress(df_[corr_main], df_[" NMR [T]"])
    line = slope * df_[corr_main] + intercept
    df[i]["NMR_const"] = df_[" NMR [T]"] - slope * df_[corr_main] - intercept + df_[" NMR [T]"].mean()

In [ ]:
for i,df_ in enumerate(df):
    print(f"run{i+1:02d}: Mean: {df_['NMR_const'].mean()}, Std: {df_['NMR_const'].std()}")

In [ ]:
df[9].head()

In [ ]:
df[8].tail()

In [ ]:
fig = scatter2d(df[9].index, df[9].NMR_const, inline=True)

In [ ]:
fig = scatter2d(2*[df[9].index], [df[9][" NMR [T]"],df[9].NMR_const], inline=True)